In [ ]:
import random
import math
import time
from openpyxl import Workbook

# Define the recipes as a list from 1 to 10
recipes = list(range(1, 11))

# Define the recipe eligibility for each factory
factory_eligibility = {
    'F1': random.sample(recipes, int(0.5 * len(recipes))),  # 50% of total recipes are eligible for F1
    'F2': random.sample(recipes, int(0.7 * len(recipes))),  # 70% of total recipes are eligible for F2
    'F3': recipes  # 100% recipes are eligible for F3
}

# Generate random orders with 1-4 recipes
def generate_orders(num_orders):
    orders = []
    for n in range(num_orders):
        recipe_ids = random.sample(recipes, random.randint(1, 4))
        orders.append({'id': len(orders) + 1, 'recipe_ids': recipe_ids})
    return orders

# Get the eligible factories for each order based on the recipes
def get_eligible_factories(order):
    eligible_factories = []
    for factory, eligible_recipes in factory_eligibility.items():
        if all(recipe_id in eligible_recipes for recipe_id in order['recipe_ids']):
            eligible_factories.append(factory)
    return eligible_factories

# Generate sample orders for LD5 and LD3
num_orders = 20
orders_ld5 = generate_orders(num_orders)
orders_ld3 = generate_orders(num_orders)

# Add eligible factories to each order
for order in orders_ld5 + orders_ld3:
    order['eligible_factories'] = get_eligible_factories(order)

# Define factory capacities
factory_capacities = {
    'F1': int(num_orders / 3),  # Serve a third of total orders
    'F2': int(num_orders * 2 / 3),  # Serve two thirds of total orders
    'F3': float('inf')  # Catch-all factory with infinite capacity
}

# Initial order allocation (orders with fewer eligible factories are allocated first, as they have limited options)
def initial_allocation(orders, factory_capacities):
    allocation = {factory: [] for factory in factory_capacities}
    sorted_orders = sorted(orders, key=lambda x: len(x['eligible_factories']))
    for order in sorted_orders:
        allocated = False
        for factory in order['eligible_factories']:
            if len(allocation[factory]) < factory_capacities[factory]:
                if all(recipe_id in factory_eligibility[factory] for recipe_id in order['recipe_ids']):
                    allocation[factory].append(order)
                    allocated = True
                    break
        if not allocated:
            allocation['F3'].append(order)
    return allocation

# Calculate WMAPE site
def calculate_wmape_site(allocation_ld5, allocation_ld3):
    total_abs_diff = 0
    total_ld3_items = 0

    for factory in ['F1', 'F2']:
        recipe_counts_ld5 = {}
        recipe_counts_ld3 = {}

        for order in allocation_ld5[factory]:
            for recipe_id in order['recipe_ids']:
                recipe_counts_ld5[recipe_id] = recipe_counts_ld5.get(recipe_id, 0) + 1

        for order in allocation_ld3[factory]:
            for recipe_id in order['recipe_ids']:
                recipe_counts_ld3[recipe_id] = recipe_counts_ld3.get(recipe_id, 0) + 1
                total_ld3_items += 1

        for recipe_id in set(recipe_counts_ld5.keys()) | set(recipe_counts_ld3.keys()):
            ld5_count = recipe_counts_ld5.get(recipe_id, 0)
            ld3_count = recipe_counts_ld3.get(recipe_id, 0)
            total_abs_diff += abs(ld5_count - ld3_count)

    if total_ld3_items == 0:
        return float('inf')  # Return a large positive number as default
    else:
        wmape_site = total_abs_diff / total_ld3_items
        return wmape_site

# Calculate WMAPE global
def calculate_wmape_global(allocation_ld5, allocation_ld3):
    total_abs_diff = 0
    total_ld3_items = 0

    recipe_counts_ld5 = {}
    recipe_counts_ld3 = {}

    for factory in allocation_ld5:
        for order in allocation_ld5[factory]:
            for recipe_id in order['recipe_ids']:
                recipe_counts_ld5[recipe_id] = recipe_counts_ld5.get(recipe_id, 0) + 1

    for factory in allocation_ld3:
        for order in allocation_ld3[factory]:
            for recipe_id in order['recipe_ids']:
                recipe_counts_ld3[recipe_id] = recipe_counts_ld3.get(recipe_id, 0) + 1
                total_ld3_items += 1

    for recipe_id in set(recipe_counts_ld5.keys()) | set(recipe_counts_ld3.keys()):
        ld5_count = recipe_counts_ld5.get(recipe_id, 0)
        ld3_count = recipe_counts_ld3.get(recipe_id, 0)
        total_abs_diff += abs(ld5_count - ld3_count)

    wmape_global = total_abs_diff / total_ld3_items
    return wmape_global

# Apply Simulated Annealing
def simulated_annealing(allocation_ld3, orders_ld3, factory_capacities, num_iterations, initial_temperature, cooling_rate):
    current_allocation = {factory: orders[:] for factory, orders in allocation_ld3.items()}
    current_wmape_site = calculate_wmape_site(allocated_orders_ld5, current_allocation)
    best_allocation = current_allocation
    best_wmape_site = current_wmape_site
    temperature = initial_temperature

    for i in range(num_iterations):
        factory1, factory2 = random.sample(['F1', 'F2', 'F3'], 2)
        order1_index = random.randint(0, len(current_allocation[factory1]) - 1)
        order2_index = random.randint(0, len(current_allocation[factory2]) - 1)
        order1 = current_allocation[factory1][order1_index]
        order2 = current_allocation[factory2][order2_index]

        if all(recipe_id in factory_eligibility[factory2] for recipe_id in order1['recipe_ids']) and \
           all(recipe_id in factory_eligibility[factory1] for recipe_id in order2['recipe_ids']):

            # Perform swap
            current_allocation[factory1][order1_index], current_allocation[factory2][order2_index] = \
                current_allocation[factory2][order2_index], current_allocation[factory1][order1_index]

            # Calculate new WMAPE
            new_wmape_site = calculate_wmape_site(allocated_orders_ld5, current_allocation)

            # Accept the swap based on Simulated Annealing criteria
            if new_wmape_site < current_wmape_site or random.random() < math.exp((current_wmape_site - new_wmape_site) / temperature):
                current_wmape_site = new_wmape_site
                if current_wmape_site < best_wmape_site:
                    best_allocation = {f: orders[:] for f, orders in current_allocation.items()}
                    best_wmape_site = current_wmape_site
            else:
                # Revert swap
                current_allocation[factory1][order1_index], current_allocation[factory2][order2_index] = \
                    current_allocation[factory2][order2_index], current_allocation[factory1][order1_index]

        # Cool down the temperature
        temperature *= cooling_rate

        # Ensure F1 and F2 have exactly their capacity
        for factory in ['F1', 'F2']:
            while len(current_allocation[factory]) > factory_capacities[factory]:
                order = current_allocation[factory].pop()
                current_allocation['F3'].append(order)

            while len(current_allocation[factory]) < factory_capacities[factory]:
                if current_allocation['F3']:
                    for i, order in enumerate(current_allocation['F3']):
                        if all(recipe_id in factory_eligibility[factory] for recipe_id in order['recipe_ids']):
                            current_allocation[factory].append(order)
                            del current_allocation['F3'][i]
                            break
                else:
                    break

    return best_allocation

# Initial allocation
allocated_orders_ld5 = initial_allocation(orders_ld5, factory_capacities)
allocated_orders_ld3 = initial_allocation(orders_ld3, factory_capacities)

# Calculate initial WMAPE site and WMAPE global
initial_wmape_site = calculate_wmape_site(allocated_orders_ld5, allocated_orders_ld3)
initial_wmape_global = calculate_wmape_global(allocated_orders_ld5, allocated_orders_ld3)
print(f"Initial WMAPE site: {initial_wmape_site:.2f}")
print(f"WMAPE global: {initial_wmape_global:.2f}")

# Apply Simulated Annealing
num_iterations = 1000
initial_temperature = 100
cooling_rate = 0.99

start_time = time.time()  # Start measuring execution time

optimized_allocation_ld3 = simulated_annealing(allocated_orders_ld3, orders_ld3, factory_capacities, num_iterations, initial_temperature, cooling_rate)

end_time = time.time()  # End measuring execution time
execution_time = end_time - start_time

# Calculate improved WMAPE site
improved_wmape_site = calculate_wmape_site(allocated_orders_ld5, optimized_allocation_ld3)
print(f"Improved WMAPE site: {improved_wmape_site:.2f}")
print(f"Execution time: {execution_time:.2f} seconds")

# Export order data and allocation solutions to Excel
def export_to_excel(orders_ld5, orders_ld3, allocated_orders_ld5, allocated_orders_ld3, optimized_allocation_ld3):
    workbook = Workbook()
    sheet_ld5_orders = workbook.active
    sheet_ld5_orders.title = 'LD5 Orders'

    sheet_ld3_orders = workbook.create_sheet(title='LD3 Orders')
    sheet_allocation_ld5 = workbook.create_sheet(title='LD5 Allocation')
    sheet_allocation_ld3 = workbook.create_sheet(title='LD3 Allocation')
    sheet_optimized_allocation_ld3 = workbook.create_sheet(title='Optimized LD3 Allocation')

    order_headers = ['Order ID', 'Recipe IDs', 'Eligible Factories']
    sheet_ld5_orders.append(order_headers)
    sheet_ld3_orders.append(order_headers)

    allocation_headers = ['Factory', 'Allocated Orders', 'Recipe IDs']
    sheet_allocation_ld5.append(allocation_headers)
    sheet_allocation_ld3.append(allocation_headers)
    sheet_optimized_allocation_ld3.append(allocation_headers)

    # Write order data for LD5
    for order in orders_ld5:
        row = [order['id'], ', '.join(map(str, order['recipe_ids'])), ', '.join(order['eligible_factories'])]
        sheet_ld5_orders.append(row)

    # Write order data for LD3
    for order in orders_ld3:
        row = [order['id'], ', '.join(map(str, order['recipe_ids'])), ', '.join(order['eligible_factories'])]
        sheet_ld3_orders.append(row)

    # Write allocation data for LD5
    for factory, orders in allocated_orders_ld5.items():
        for order in orders:
            row = [factory, order['id'], ', '.join(map(str, order['recipe_ids']))]
            sheet_allocation_ld5.append(row)

    # Write allocation data for LD3
    for factory, orders in allocated_orders_ld3.items():
        for order in orders:
            row = [factory, order['id'], ', '.join(map(str, order['recipe_ids']))]
            sheet_allocation_ld3.append(row)

    # Write optimized allocation data for LD3
    for factory, orders in optimized_allocation_ld3.items():
        for order in orders:
            row = [factory, order['id'], ', '.join(map(str, order['recipe_ids']))]
            sheet_optimized_allocation_ld3.append(row)

    # Save the workbook
    workbook.save('SA.xlsx')

# Export the order data and allocation solutions to Excel
export_to_excel(orders_ld5, orders_ld3, allocated_orders_ld5, allocated_orders_ld3, optimized_allocation_ld3)

Initial WMAPE site: 0.91
WMAPE global: 0.38
